# Introduction to Pytorch Workflow


In [11]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from torch import nn  #neural networks
print(torch.__version__)

2.7.1+cu118


In [12]:
what_were_covering={
    1:'data preparations and loading', 
    2:'building a model', 
    3:'fitting a model', 
    4:'making predictions and evaluating a model', 
    5:'saving and loading a model', 
    6:'putting a model on GPU'
    }

## 1. Data Preparations and Loading

In [13]:
# creatig Known parameters
weight=0.7
bias=0.3

#create
start=0
end=1
step=0.02
X=torch.arange(start,end,step).unsqueeze(dim=1) #unsqueeze to add a dimension
Y=weight*X+bias
print(X[:10],Y[:10],len(X),len(Y))

tensor([[0.0000],
        [0.0200],
        [0.0400],
        [0.0600],
        [0.0800],
        [0.1000],
        [0.1200],
        [0.1400],
        [0.1600],
        [0.1800]]) tensor([[0.3000],
        [0.3140],
        [0.3280],
        [0.3420],
        [0.3560],
        [0.3700],
        [0.3840],
        [0.3980],
        [0.4120],
        [0.4260]]) 50 50


### Spliting the dataset to train and test datasets

In [14]:
#create train and test split
train_split=int(0.8*len(X))
X_train, Y_train=X[:train_split], Y[:train_split]
X_test, Y_test=X[train_split:], Y[train_split:]